In [ ]:
access_key = 'xxx'
secret_key = 'xxx'
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

# If you are using Auto Loader file notification mode to load files, provide the AWS Region ID.
aws_region = "ap-us-east-1"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")


In [ ]:
AWS_S3_BUCKET = 'manan-airflow-bucket'
MOUNT_NAME = '/mnt/mount_s3'


In [ ]:
SOURCE_URL = "s3a://%s:%s@%s" %(access_key, secret_key, AWS_S3_BUCKET)

In [ ]:
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[9]: True

In [ ]:
%fs ls '/mnt/mount_s3'

path,name,size,modificationTime
dbfs:/mnt/mount_s3/athlete_events.csv,athlete_events.csv,23724678,1688839732000
dbfs:/mnt/mount_s3/athletes.csv,athletes.csv,6071413,1688839737000


In [ ]:
file_location = "dbfs:/mnt/mount_s3/athletes.csv"
athletes_df = spark.read.csv(file_location, inferSchema = True, header= True)
athletes_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- team: string (nullable = true)



In [ ]:
file_location_1 = "dbfs:/mnt/mount_s3/athlete_events.csv"
athlete_events = spark.read.csv(file_location_1, inferSchema = True, header= True)
athlete_events.printSchema()

root
 |-- athlete_id: integer (nullable = true)
 |-- games: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- season: string (nullable = true)
 |-- city: string (nullable = true)
 |-- sport: string (nullable = true)
 |-- event: string (nullable = true)
 |-- medal: string (nullable = true)



In [ ]:
# Snowflake Connection credentials
sfOptions = {
  "sfUrl": "xxx",
  "sfUser": "xxx",
  "sfPassword": "xxx",
  "sfDatabase": "OLYMPIC_DB",
  "sfSchema": "PUBLIC",
  "sfWarehouse": "COMPUTE_WH"
}
     

In [ ]:
table_name = "athletes"

# Write the DataFrame to Snowflake
athletes_df.write.format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", table_name) \
    .mode("overwrite") \
    .options(header=True) \
    .save()

In [ ]:
# Specify the Snowflake table name
table_name = "athlete_events"

# Write the DataFrame to Snowflake
athlete_events.write.format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", table_name) \
    .mode("overwrite") \
    .options(header=True) \
    .save()
     

In [ ]:
# Specify the Snowflake table name
table_name = "athlete_events"

# Read the data written by the previous cell back.
df = spark.read \
  .format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", table_name) \
  .load()
 
df.show(10)

+----------+-----------+----+------+--------------+------------+--------------------+------+
|ATHLETE_ID|      GAMES|YEAR|SEASON|          CITY|       SPORT|               EVENT| MEDAL|
+----------+-----------+----+------+--------------+------------+--------------------+------+
|    120259|2016 Summer|2016|Summer|Rio de Janeiro|Table Tennis|Table Tennis Wome...|    NA|
|    120260|1956 Summer|1956|Summer|     Melbourne|      Boxing|Boxing Men's Welt...|Silver|
|    120261|1968 Summer|1968|Summer|   Mexico City|   Athletics|Athletics Men's D...|    NA|
|    120262|1992 Summer|1992|Summer|     Barcelona|   Athletics|Athletics Women's...|    NA|
|    120262|2000 Summer|2000|Summer|        Sydney|   Athletics|Athletics Women's...|    NA|
|    120263|1964 Winter|1964|Winter|     Innsbruck|        Luge|  Luge Men's Singles|    NA|
|    120264|1904 Summer|1904|Summer|     St. Louis|   Wrestling|Wrestling Men's L...|Bronze|
|    120265|1948 Summer|1948|Summer|        London|      Hockey| Hocke